In [1]:
import requests
import pandas as pd
import lxml.etree as ET
from bs4 import BeautifulSoup as soup
import unicodedata
from tqdm.notebook import tqdm
from time import sleep
import time

In [39]:
def dnb_sru(query):
    
    base_url = "https://services.dnb.de/sru/dnb"
    params = {'recordSchema' : 'MARC21-xml',
          'operation': 'searchRetrieve',
          'version': '1.1',
          'maximumRecords': '100',
          'query': query
         }
    r = requests.get(base_url, params=params)
    xml = soup(r.content, features="xml")
    records = xml.find_all('record', {'type':'Bibliographic'})
    treffer = xml.find_all("numberOfRecords")[0].text
    treffer = int(treffer)
    loops = int(treffer/100)
    print(treffer)
    
     
       
    if len(records) < 100:
        
        return xml
    
    else:
    
        #Define XML-header and footer: 
        header = f'''<?xml version="1.0" encoding="utf-8"?><searchRetrieveResponse xmlns="http://www.loc.gov/zing/srw/"> 
            <version>1.1</version><numberOfRecords>{str(treffer)}</numberOfRecords>
            <records><record><recordSchema>MARC21-xml</recordSchema><recordPacking>xml</recordPacking><recordData>'''      
        footer = f'''</recordData></record></records><echoedSearchRetrieveRequest><version>1.1</version>
            <query>{query}</query><xQuery xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xsi:nil="true"/>
            <maximumRecords>100</maximumRecords><recordSchema>MARC21-xml</recordSchema></echoedSearchRetrieveRequest>
            </searchRetrieveResponse>'''
    
        num_results = 100
        i = 101
        pbar = tqdm(total=loops)
        while num_results == 100:
            
            params.update({'startRecord': i})
            r = requests.get(base_url, params=params)
            xml = soup(r.content, features="xml")
            new_records = xml.find_all('record', {'type':'Bibliographic'})
            records+=new_records
            i+=100
            num_results = len(new_records)
            pbar.update()
            
        recordlist = ""
        for record in records:
            recordlist += str(record)
                  
        xmlplus = header + recordlist + footer
        
        return xmlplus

In [40]:
#myquery = dnb_sru('hss=(nat* and diss*) and jhr>=1900 and jhr<=1970')
myquery = dnb_sru('tit=Klimawandel')

3923


  0%|          | 0/39 [00:00<?, ?it/s]

In [42]:
# simple dump as string
with open('klima_dump.xml', 'w', encoding="utf-8") as f:
    f.write(myquery)

In [50]:
# less simple dump in xml-structure

# WARNING: NOT AT ALL GREAT TO USE - adds extra whitespaces in strings!!!

for element in myquery:
    with open('file.xml', 'a', encoding="utf-8") as f:
        f.write(element.prettify())